# Topic Modeling
2023-02-09
1. Crawling News
2. Preprocessing
3. Build Term-Document Matrix
4. Topic modeling
5. Visualization

```
1. 네이버에서 뉴스 기사를 크롤링
2. 기본적인 전처리 이후 Term-document Matrix를 만들고 이를 non-negative factorization을 이용해 행렬 분해를 하여 Topic modeling을 수행
3. t-distributed stochastic neighbor embedding(T-SNE) 기법을 이용해 Topic별 시각화를 진행
```

In [1]:
!pip install newspaper3k
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=0d4004644d256863c18a2c5d483b812961ca3918a744cc1cc3f3b8f08fae20e1
  Stored in directory: /root/.cache/pip/wheels/99/74/83/8fac1c8d9c648cfabebbbffe97a889f6624817f3aa0bbe6c09
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3354 sha256=ffb7c1b602978134ba12e0790faae749f319aa7a82b3141c88e11e

In [2]:
# 크롤링에 필요한 패키지 설치
from bs4 import BeautifulSoup
from newspaper import Article
from time import sleep
from time import time
from dateutil.relativedelta import relativedelta
from datetime import datetime
from multiprocessing import Pool
import json
import requests
import re
import sys

In [3]:
def crawl_news(query: str=None, crawl_num: int=1000, workers: int=4):
    '''뉴스 기사 텍스트가 담긴 list를 반환합니다.

    Keyword arguments:
    query -- 검색어 (default None)
    crawl_num -- 수집할 뉴스 기사의 개수 (defualt 1000)
    workers -- multi-processing시 사용할 thread의 개수 (default 4)
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
    articleList = []
    crawled_url = set()
    keyboard_interrupt = False
    t = time()
    idx = 0
    page = 1

    
    # 서버에 url 요청의 결과를 선언
    res = requests.get(url.format(query))
    sleep(0.5)
    # res를 parsing할 parser를 선언
    bs = BeautifulSoup(res.text, 'html.parser')
    
    with Pool(workers) as p:
        while idx < crawl_num:            
            table = bs.find('ul', {'class': 'list_news'})
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
            area_list = [li.find('div', {'class':'news_area'}) for li in li_list]
            a_list = [area.find('a', {'class':'news_tit'}) for area in area_list]
            
            for n in a_list[:min(len(a_list), crawl_num-idx)]:
                articleList.append(n.get('title'))
                idx += 1
            page += 1

            pages = bs.find('div', {'class': 'sc_page_inner'})
            next_page_url = [p for p in pages.find_all('a') if p.text == str(page)][0].get('href')

            req = requests.get('https://search.naver.com/search.naver' + next_page_url)
            bs = BeautifulSoup(req.text, 'html.parser')
    return articleList

```
'구글'을 포함한 뉴스 기사 1000개의 제목을 크롤링
```

In [4]:
query = '구글'

articleList = crawl_news(query)

In [5]:
articleList[:10]

['구글, \'챗GPT 대항마\' 발표…"테스트 거쳐 수주 안에 공개"(종합)',
 "구글, 챗GPT 대항마 '바드'AI 발표...'수주일 내 공개'",
 'MS, ‘빙’ 검색에 ‘챗GPT’ 통합···구글과 ‘검색전쟁’',
 "MS '빙' 챗GPT DNA 심는다…구글 검색 뛰어넘나",
 'MS 검색엔진에 챗GPT 탑재…“구글 잡겠다”',
 "'챗GPT와 바드, MS와 구글'…누가 최종 승자가 될까",
 'MS, 챗GPT 탑재한 ‘검색엔진 빙’ 내놔…구글 심장까지 겨냥',
 '챗GPT 대항마 예고, 구글 가세…시장 선점 경쟁 속 네카오 대응은',
 '구글·모질라, 독자 엔진 기반 iOS용 브라우저 준비',
 '“우주망원경 묻자, 9세 눈높이 맞춰 대답”… 구글 AI챗봇도 나온다']

```
🔥 태거(tagger)를 이용해 한글 명사와 알파벳만을 추출해서 term-document matrix (tdm) 생성
```

참고: [konlpy: morph analyzer](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

## Preprocessing

In [32]:
from konlpy.tag import Okt
from collections import Counter
import json

# Okt 형태소 분석기 선언
t = Okt()

words_list_ = []
vocab = Counter()
tag_set = set(['Noun', 'Alpha'])
stopwords = set(['글자'])

for i, article in enumerate(articleList):
    if i % 100 == 0:
        print(i)
    
    # tagger를 이용한 품사 태깅
    words = t.pos(article, norm=True, stem=True)

    # condition 1: 명사와 알파벳 tag를 가진 단어
    # condition 2: 철자 길이가 2이상인 단어 
    # condition 3: stopwords에 포함되지 않는 단어     
    if i< 10:
      print(words[0])
    words = [word for word, tag in words if tag == 'Noun' or tag == "Alpha" and word not in stopwords]

    vocab.update(words)
    words_list_.append((words, article))
    
vocab = sorted([w for w, freq in vocab.most_common(10000)])
word2id = {w: i for i, w in enumerate(vocab)}
words_list = []
for words, article in words_list_:
    words = [w for w in words if w in word2id]
    if len(words) > 10:
        words_list.append((words, article))
        
del words_list_

0
('구글', 'Noun')
('구글', 'Noun')
('MS', 'Alpha')
('MS', 'Alpha')
('MS', 'Alpha')
("'", 'Punctuation')
('MS', 'Alpha')
('챗', 'Noun')
('구글', 'Noun')
('“', 'Foreign')
100
200
300
400
500
600
700
800
900


## Build document-term matrix

```
document-term matrix 생성
document-term matrix = (문서 개수 x 단어 개수)의 Matrix
```

참고: [Document-Term Matrix](https://wikidocs.net/24559)

In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

dtm = np.zeros((len(words_list), len(vocab)), dtype=np.float32)
for i, (words, article) in enumerate(words_list):
    for word in words:
        dtm[i, word2id[word]] += 1
        
dtm = TfidfTransformer().fit_transform(dtm)

```
document-term matrix를 non-negative factorization(NMF)을 이용해 행렬 분해
```
**Non-negative Factorization**
```
NMF는 주어진 행렬 non-negative matrix X를 non-negative matrix W와 H로 행렬 분해하는 알고리즘이다.
```
참고: [Non-negative Matrix Factorization](https://angeloyeo.github.io/2020/10/15/NMF.html)

## Topic modeling

In [34]:
# Non-negative Matrix Factorization
from sklearn.decomposition import NMF

K=5
nmf = NMF(n_components=K, alpha=0.1)

```
W는 document length x K, H는 K x term length의 차원을 갖고 있다.
W의 하나의 row는 각각의 feature에 얼만큼의 가중치를 줄 지에 대한 weight이며
H의 하나의 row는 하나의 feature를 나타낸다.
```

In [35]:
W = nmf.fit_transform(dtm)
H = nmf.components_

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:1422: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


```
하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력하면 다음과 같다.
```

In [36]:
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
    print()

0th topic
주 특징 AI 출시 엔젤 유 관련 소식 상하 급등 챗봇 MDS 엔비디아 부각 구글 바드 테크 파트너 공식 대항 
1th topic
삼성 퀄컴 XR 메타 동맹 전자 버스 애플 결의 구글 삼국시대 갤럭시 예고 미래 오늘 팩 생태계 변화 모바일 박사 
2th topic
뮤직 지니 생 LM 개발 강세 음악 주가 지능 인공 AI 구글 공개 영향 대감 휘파람 주목 수혜 장 ET 
3th topic
GPT 챗 AI 바드 MS 대항 마 챗봇 전쟁 구글 발표 불 경쟁 공개 본격 테스트 빅 테크 공식 대화 
4th topic
버디 샷 블록 게임 체인 골프 글로벌 보라 플레이 시작 메타 사전예약 사전 예약 구글플레이 캐 주얼 돌입 확충 접근성 


```
W에서 하나의 Topic (W의 n번째 column)에 접근해서 해당 topic에 대해 값이 가장 높은 3개의 뉴스 기사 제목을 출력하면 다음과 같다.
```

In [37]:
for k in range(K):
    print(f"==={k}th topic===")
    for index in W[:, k].argsort()[::-1][:3]:
        print(words_list[index][1])
    print('\n')

===0th topic===
[특징주] 유엔젤, 구글 AI챗봇 '바드' 출시 소식에 상한가
[7일 특징주] 구글, 챗GPT 대항마 '바드' 출시 소식에 관련주 급등…유엔젤 등 6개 종목 상한가
[7일 특징주] 구글, 챗GPT 대항마 '바드' 출시 소식에 관련주 급등…유엔젤 등 6개 종목 상한가


===1th topic===
삼성전자, 퀄컴·구글과 ‘XR 동맹’ 결의… 메타·애플과 ‘메타버스 삼국시대’ 예고
삼성전자, 퀄컴·구글과 ‘XR 동맹’ 결의… 메타·애플과 ‘메타버스 삼국시대’ 예고
삼성전자, 퀄컴·구글과 ‘XR 동맹’ 결의… 메타·애플과 ‘메타버스 삼국시대’ 예고


===2th topic===
지니뮤직 주가 강세, 구글 음악생성 AI '뮤직LM' 개발
지니뮤직 주가 강세, 구글 음악생성 AI '뮤직LM' 개발
지니뮤직, 구글 인공지능 뮤직LM 개발 소식에 주가 강세


===3th topic===
챗GPT가 불붙인 빅테크 AI 전쟁...구글, MS보다 먼저 AI 챗봇 '바드' 꺼내 들었다
[돈이 되는 뉴스] 구글, '챗GPT 대항마' 바드 공식 발표… 불붙은 AI챗봇 전쟁
구글 '바드' 출시… MS 챗GPT 추격 "불붙은 AI 챗봇 전쟁"


===4th topic===
메타보라, 블록체인 골프 게임 '버디샷' 구글 플레이 사전예약 시작
메타보라, 블록체인 골프게임 '버디샷' 구글 사전예약 시작
블록체인 골프 게임 버디샷, 구글플레이 글로벌 사전 예약 시작




```
2번째 토픽에 대해 가장 높은 가중치를 갖는 제목 5개를 출력
```

In [38]:
#TODO
for i in W[:, 2].argsort()[::-1][:5]:
  print(words_list[i][1])

지니뮤직 주가 강세, 구글 음악생성 AI '뮤직LM' 개발
지니뮤직 주가 강세, 구글 음악생성 AI '뮤직LM' 개발
지니뮤직, 구글 인공지능 뮤직LM 개발 소식에 주가 강세
[특징주] 지니뮤직, 구글 음악 생성 AI '뮤직LM' 공개에 강세
지니뮤직 주가 휘파람...구글, 생성 인공지능 뮤직LM 개발 주목


`t-SNE를 이용한 Topic별 시각화`

**t-SNE**

```
t-Stochastic Neighbor Embedding(t-SNE)은 PCA기법과 함께 고차원의 벡터를 
저차원(2~3차원) 벡터로 데이터간 구조적 특징을 유지하며 축소를 하는 방법 중 하나이다.

주로 고차원 데이터의 시각화를 위해 사용된다.
```

참고: [lovit: t-SNE](https://lovit.github.io/nlp/representation/2018/09/28/tsne/#:~:text=t%2DSNE%20%EB%8A%94%20%EA%B3%A0%EC%B0%A8%EC%9B%90%EC%9D%98,%EC%9D%98%20%EC%A7%80%EB%8F%84%EB%A1%9C%20%ED%91%9C%ED%98%84%ED%95%A9%EB%8B%88%EB%8B%A4.)

참고: [ratsgo: t-SNE](https://ratsgo.github.io/machine%20learning/2017/04/28/tSNE/)

## Visualization

In [39]:
from sklearn.manifold import TSNE

# n_components = 차원 수
tsne = TSNE(n_components=2, init='pca', verbose=1)

# W matrix에 대해 t-sne를 수행
W2d = tsne.fit_transform(W)

# 각 뉴스 기사 제목마다 가중치가 가장 높은 topic을 저장
topicIndex = [v.argmax() for v in W]

/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/manifold/_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 208 samples in 0.001s...
[t-SNE] Computed neighbors for 208 samples in 0.006s...
[t-SNE] Computed conditional probabilities for sample 208 / 208
[t-SNE] Mean sigma: 0.027638
[t-SNE] KL divergence after 250 iterations with early exaggeration: 54.734871
[t-SNE] KL divergence after 1000 iterations: 0.100241


In [40]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article for words, article in words_list],
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color'
)

# interaction
p.legend.location = "top_left"
hover = p.select({'type': HoverTool})
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)